In [1]:
import cv2 as cv
import numpy as np
import keyboard
import time

import Controlling_thymio
import global_planning
import vision

In [ ]:
# Global constants here

IMAGE_WIDTH = 640 # width of the video feed

IMAGE_HEIGHT = 480 # height of the video feed

CAMERA_FPS = 30 # fps of the video feed

KEYBOARD_INPUT = "enter" # keyboard input to start

CAMERA_REFRESH_TIME = 30 # number of frame thrown away to allow the camera to focus in the meantime

KP_LINEAR = 5 # linear proportional gain in PI controller

KI_LINEAR = 0.5 # linear integral gain in PI controller

KP_ANGULAR = 15 # angular proportional gain in PI controller

KI_ANGULAR = 1.5 # angular linear gain in PI controller

PATH_DELTA = 7 # acccepted difference in pixels between the actual robot's position and its goal

ANGULAR_DELTA = 0.15 # accepted difference in radian between the actual robot's angle and its goal

TURNING_SPEED = 100 # speed of the wheel when turning

STRAIGHT_SPEED = 150 # speed of the wheel when going straight

In [4]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
import Controlling_thymio

node = await client.wait_for_node()

await node.lock()
aw(node.lock())

await node.wait_for_variables({"acc"})

In [ ]:
# 0 is laptop webcam, 1 is USB camera (!COMPUTER DEPENDENT!)
cap = cv.VideoCapture(0)

init = False # boolean set to True on keyboard input to start initialisation
is_init = False # boolean set to True once initialisation has succeeded
black_frame_remaining = 0 # numbers of frames to throw away
being_kidnapped = False # boolean set to True while the robot is being kidnapped
error_linear = [] # array of all linear errors
error_angle = [] # array of all angular errors
initial_turn = True # boolean set to True while the first orientation is not done

# variable initialized at False irrelevant of wether it actually is
camera_hidden = False

# create a robot instance
robot_instance = global_planning.Robot()

# if unable to connect to the camera
if not (cap.isOpened()):
    print("Could not open video device")
else:
    while(True):
        # get the input frame by frame (shape (480,640,3))
        ret, frame = cap.read() 
        cap.set(cv.CAP_PROP_FPS, CAMERA_FPS) 
        frame = cv.resize(frame, (IMAGE_WIDTH,IMAGE_HEIGHT), interpolation=cv.INTER_CUBIC) 

        # wait until keyboard input to initialize vision
        if keyboard.is_pressed(KEYBOARD_INPUT):
            init = True

        # reinitialize the robot after a kidnapping
        if being_kidnapped: 
            if not Controlling_thymio.kidnapping(node):
                
                # to give time to the kidnapper to get out of the frame
                time.sleep(3)

                # reinitialize the starting position of the robot
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                start_x, start_y, alpha, width = vision.get_robot_position(fop)

                # reinitialize the global planning
                converted_obstacles = [[tuple(arr) for arr in sublist] for sublist in obstacles]
                converted_obstacles = global_planning.obstacle_dictionnary(converted_obstacles)
                goal = (objective_x, objective_y)
                robot_instance.update_coordinates(start_x, start_y, alpha, width)
                obstacles_named = global_planning.naming_points(converted_obstacles, robot_instance, goal)
                adg_list = global_planning.creating_adjacency_dictionnary(converted_obstacles, robot_instance, goal)
                path = global_planning.finding_path(adg_list, obstacles_named)

                is_init = True
                being_kidnapped = False
        
        # initialize 
        if init == True:

            try:
                # vision: start, obstacles and goal coordinates
                original_coordinates, new_coordinates = vision.get_fop_coordinates(frame)
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                start_x, start_y, alpha, width = vision.get_robot_position(fop)
                obstacles = vision.get_obstacles(fop, width)
                objective_x, objective_y = vision.get_objective(fop)

                # global planning
                converted_obstacles = [[tuple(arr) for arr in sublist] for sublist in obstacles]
                converted_obstacles = global_planning.obstacle_dictionnary(converted_obstacles)
                goal = (objective_x,objective_y)
                robot_instance.update_coordinates(start_x, start_y, alpha,width)
                obstacles_named = global_planning.naming_points(converted_obstacles,robot_instance,goal)
                adg_list = global_planning.creating_adjacency_dictionnary(converted_obstacles,robot_instance,goal)
                path = global_planning.finding_path(adg_list,obstacles_named)
                print(path)
                is_init = True
            except Exception as e:
                print("Initialisation failed: " + str(e))

            init = False

        if is_init:  
        ################################## MOTION CONTROL HERE ################################## 
            
            # get out of the loop if the robot is being kidnapped
            if Controlling_thymio.kidnapping(node):
                being_kidnapped = True
                is_init = False
                continue

            fop = vision.get_fop(frame, original_coordinates, new_coordinates)
            center_x, center_y, alpha, width = vision.get_robot_position(fop)
            
            # camera was covered, now uncovered
            if center_x != 0 and center_y != 0 and camera_hidden:
                camera_hidden = False
                black_frame_remaining = CAMERA_REFRESH_TIME # to give it time to gain focus

            # camera uncovered and usable -> navigation using vision
            elif black_frame_remaining <= 0:
                fop = cv.circle(fop, (center_x,center_y), radius=10, color=(0, 255, 0), thickness=-1)

                # ADD PROXIMITY AVOIDANCE

                start_point = np.array([obstacles_named[path[0]][0],obstacles_named[path[0]][1]])
                end_point = np.array([obstacles_named[path[1]][0],obstacles_named[path[1]][1]])
                
                # traget point after the actual end point
                try:
                    next_point = np.array([obstacles_named[path[2]][0],obstacles_named[path[2]][1]])
                except:
                    next_point = [0, 0]

                robot_center = np.array([center_x, center_y])

                # do the initial orientation
                if initial_turn:
                    
                    # turning
                    error_angle = Controlling_thymio.get_angular_error(end_point, next_point, alpha)
                    left_wheel_speed = -1 * np.sign(error_angle) * TURNING_SPEED
                    right_wheel_speed = np.sign(error_angle) * TURNING_SPEED

                    # first turn is done
                    if Controlling_thymio.get_angular_error(end_point, next_point, alpha) < ANGULAR_DELTA:
                        initial_turn = False

                # check if the next point as been reached
                if Controlling_thymio.reached_linear_target(end_point, robot_center, PATH_DELTA):
                    
                    # if the last point has been reached
                    if next_point[0] == 0 and next_point[1] == 0:
                        print("Target reached")
                    else:
                        if Controlling_thymio.get_angular_error(end_point, next_point, alpha) < ANGULAR_DELTA:
                            # finished turning, ready to go straight
                            path = path[1:]

                        else:
                            # turning
                            error_angle = Controlling_thymio.get_angular_error(end_point, next_point, alpha)
                            left_wheel_speed = -1 * np.sign(error_angle) * TURNING_SPEED
                            right_wheel_speed = np.sign(error_angle) * TURNING_SPEED
                else:
                    # does not do the straight control if doing the initial turning
                    if not initial_turn:
                        # going straight
                        error_linear.append(Controlling_thymio.get_linear_error(start_point, end_point,robot_center))
                        error_angle.append(Controlling_thymio.get_angular_error(start_point, end_point,2.65))

                        left_wheel_speed = STRAIGHT_SPEED - Controlling_thymio.PI_controller(error_angle, KP_ANGULAR, KI_ANGULAR) + Controlling_thymio.PI_controller(error_linear, KP_LINEAR, KI_LINEAR)
                        right_wheel_speed = STRAIGHT_SPEED + Controlling_thymio.PI_controller(error_angle, KP_ANGULAR, KI_ANGULAR) - Controlling_thymio.PI_controller(error_linear, KP_LINEAR, KI_LINEAR)
                    else:
                        pass
            else:
                # camera was uncovered, now covered
                if center_x == 0 and center_y == 0:
                    camera_hidden = True
                else:
                    # set the first frames after uncovering to black to allow time to focus -> navigation using extended Kalman filter
                    fop = cv.cvtColor(fop, cv.COLOR_BGR2GRAY)
                    _, fop = cv.threshold(fop,255,255,cv.THRESH_BINARY)
                    black_frame_remaining -= 1

                # DO KALMAN NAVIGATION HERE

                # ADD PROXIMITY AVOIDANCE

            # draw everything
            if not camera_hidden and black_frame_remaining <= 0:
                cv.circle(fop, (objective_x, objective_y), radius=10, color=(255, 0, 0), thickness=-1)
                cv.circle(fop, (start_x, start_y), radius=10, color=(0, 0, 255), thickness=-1)
                for obstacle in obstacles:
                    for vertice in obstacle:
                        cv.circle(fop, (vertice[0],vertice[1]), radius=10, color=(0, 255, 0), thickness=-1)
                for i in range(len(path[:-1])):
                    cv.line(fop,(obstacles_named[path[i]][0],obstacles_named[path[i]][1]),(obstacles_named[path[i+1]][0],obstacles_named[path[i+1]][1]),(255,0,0),5)
    

        #########################################################################################
        else:
            # to output the raw feed
            fop = frame

        cv.imshow('Raw',frame)
        cv.imshow('Processed',fop)

        await client.sleep(0.001)
        
        # Close all windows and get out of the loop if ESC is pressed
        if cv.waitKey(1) == 27:
            cv.destroyAllWindows()
            break

Initialisation failed: cannot access local variable 'further_vertices' where it is not associated with a value
Initialisation failed: cannot access local variable 'further_vertices' where it is not associated with a value
['R', 'G']
['R', 'G']
['R', 'G']
Initialisation failed: cannot access local variable 'further_vertices' where it is not associated with a value
Initialisation failed: cannot access local variable 'further_vertices' where it is not associated with a value
Initialisation failed: cannot access local variable 'further_vertices' where it is not associated with a value
Initialisation failed: cannot access local variable 'further_vertices' where it is not associated with a value
Initialisation failed: cannot access local variable 'further_vertices' where it is not associated with a value
Initialisation failed: cannot access local variable 'further_vertices' where it is not associated with a value
Initialisation failed: cannot access local variable 'further_vertices' where it